In [2]:
from google.colab import files
uploaded = files.upload()

Saving Books.csv to Books.csv


In [3]:
from google.colab import files
uploaded = files.upload()

Saving Ratings.csv to Ratings.csv


In [4]:
import pandas as pd

# Load datasets
books = pd.read_csv("Books.csv")
ratings = pd.read_csv("Ratings.csv")

print("Books Columns:", books.columns)
print("Ratings Columns:", ratings.columns)

Books Columns: Index(['Unnamed: 0', 'ISBN', 'book_id', 'Publication Year', 'Author', 'Title',
       'AvgRating', 'Image-URL', 'Image-URL-S'],
      dtype='object')
Ratings Columns: Index(['book_id', 'user_id', 'rating'], dtype='object')


In [5]:
# Rename for easy access
books.rename(columns={
    "Publication Year": "year",
    "Author": "author",
    "Title": "title",
    "AvgRating": "average_rating",
    "Image-URL": "image_url"
}, inplace=True)

books.head()

,Unnamed: 0,ISBN,book_id,year,author,title,average_rating,image_url,Image-URL-S
0,0,0195153448,1,2008,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",4.34,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,1,0002005018,2,1997,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,4.44,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,2,0060973129,3,2005,Stephenie Meyer,"Twilight (Twilight, #1)",3.57,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,3,0374157065,4,1960,Harper Lee,To Kill a Mockingbird,4.25,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,4,0393045218,5,1925,F. Scott Fitzgerald,The Great Gatsby,3.89,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [6]:
# Count how many ratings each book has
rating_counts = ratings.groupby("book_id")["rating"].count().reset_index()
rating_counts.rename(columns={"rating": "ratings_count"}, inplace=True)

# Merge with books
books = books.merge(rating_counts, on="book_id", how="left")
books["ratings_count"] = books["ratings_count"].fillna(0)

In [7]:
# Filter books with at least 50 ratings
min_ratings = 50
popular_books = books[books["ratings_count"] >= min_ratings]

# Weighted rating calculation
C = books["average_rating"].mean()

def weighted_rating(x, m=min_ratings, C=C):
    v = x["ratings_count"]
    R = x["average_rating"]
    return (v/(v+m) * R) + (m/(v+m) * C)

popular_books["weighted_rating"] = popular_books.apply(weighted_rating, axis=1)

/tmp/ipython-input-1068910502.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popular_books["weighted_rating"] = popular_books.apply(weighted_rating, axis=1)


In [8]:
top_30_books = popular_books.sort_values(by="weighted_rating", ascending=False).head(30)

# Select only required columns
top_30_books = top_30_books[["year", "author", "title", "average_rating", "image_url"]]

top_30_books

,year,author,title,average_rating,image_url
3627,2005,Bill Watterson,The Complete Calvin and Hobbes,4.82,https://images.gr-assets.com/books/1473064526m...
861,2014,Brandon Sanderson,"Words of Radiance (The Stormlight Archive, #2)",4.77,https://images.gr-assets.com/books/1391535251m...
3274,2003,"J.K. Rowling, Mary GrandPré","Harry Potter Boxed Set, Books 1-5 (Harry Potte...",4.77,https://s.gr-assets.com/assets/nophoto/book/11...
4482,1996,Bill Watterson,It's a Magical World: A Calvin and Hobbes Coll...,4.75,https://images.gr-assets.com/books/1437420710m...
6360,1996,Bill Watterson,There's Treasure Everywhere: A Calvin and Hobb...,4.74,https://s.gr-assets.com/assets/nophoto/book/11...
421,1998,J.K. Rowling,"Harry Potter Boxset (Harry Potter, #1-7)",4.74,https://images.gr-assets.com/books/1392579059m...
8853,1993,Francine Rivers,Mark of the Lion Trilogy,4.76,https://images.gr-assets.com/books/1349032180m...
7946,2002,"Anonymous, Lane T. Dennis, Wayne A. Grudem",ESV Study Bible,4.76,https://images.gr-assets.com/books/1410151002m...
6589,1990,Bill Watterson,The Authoritative Calvin and Hobbes: A Calvin ...,4.73,https://images.gr-assets.com/books/1391114055m...
3752,2005,J.K. Rowling,"Harry Potter Collection (Harry Potter, #1-6)",4.73,https://images.gr-assets.com/books/1328867351m...


In [9]:
# Save the final top 30 dataset to CSV
top_30_books.to_csv("top_30_books.csv", index=False)

print("✅ Top 30 books saved as top_30_books.csv")

✅ Top 30 books saved as top_30_books.csv
